In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

train_dir = 'C:/Users/Arun/Desktop/ENPM 673/Project 6/train/train'
test_dir = 'C:/Users/Arun/Desktop/ENPM 673/Project 6/test1/test1'
image_size=50 #Resize the images as not all are of the same size
learning_rate=1e-3
#Creating a 2 Layer Neural Network model 
MODEL_NAME = 'catsvsdogs-{}-{}'.format(learning_rate,'2conv-nn')

In [2]:
#Preprocessing data
def label_img(img):
    #All images in the testing are named as dog.9763.jpg where .jpg represents -1 and the like.Splitting the image labels with respect to period '.'
    word_label = img.split ('.')[-3]
    if word_label == 'cat':return[1,0] #All cat images
    elif word_label == 'dog':return[0,1] #All dog images

In [3]:
def create_training_data():
    training_data = []
    for img in tqdm(os.listdir(train_dir)):
        label=label_img(img)
        path =os.path.join(train_dir,img)
        img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(image_size,image_size))#tuple to resize the images in the path
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('training_data.npy',training_data)
    return training_data

In [4]:
def testing_data():
    testing_data=[]
    for img in tqdm(os.listdir(test_dir)):
        path =os.path.join(test_dir,img)
        img_num=img.split('.')[0]
        img =cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(image_size,image_size))
        testing_data.append([np.array(img),img_num])
    np.save('testing_data.npy',testing_data)
    return testing_data

In [5]:
training_data=create_training_data()

100%|██████████| 25000/25000 [00:28<00:00, 890.60it/s]


In [6]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


convnet = input_data(shape=[None, image_size, image_size, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=learning_rate, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet,tensorboard_dir ='log')


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [7]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('Model Loaded!')

In [8]:
train = training_data[:-500]
test = training_data[-500:]

In [9]:
X = np.array([i[0] for i in train]).reshape(-1,image_size,image_size,1) #features
Y = [i[1] for i in train]   #labels

test_x = np.array([i[0] for i in train]).reshape(-1,image_size,image_size,1)    
test_y = [i[1] for i in test]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 382  | total loss: 11.66915 | time: 57.099s
| Adam | epoch: 001 | loss: 11.66915 - acc: 0.4932 -- iter: 24448/24500


Exception in thread Thread-11:
Traceback (most recent call last):
  File "D:\AnacondaPython\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\AnacondaPython\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "D:\AnacondaPython\lib\site-packages\tflearn\data_flow.py", line 187, in fill_feed_dict_queue
    data = self.retrieve_data(batch_ids)
  File "D:\AnacondaPython\lib\site-packages\tflearn\data_flow.py", line 222, in retrieve_data
    utils.slice_array(self.feed_dict[key], batch_ids)
  File "D:\AnacondaPython\lib\site-packages\tflearn\utils.py", line 187, in slice_array
    return X[start]
IndexError: index 500 is out of bounds for axis 0 with size 500

